# Analyze Twenty One Pilots Lyrics with Python
### Code and process based on CodeCademy's "Analyze Taylor Swift Lyrics with Python".


In [ ]:
%matplotlib inlines

import pandas as pd
import string
import seaborn as sns
import matplotlib.pyplot as plt
import collections
import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

### 1. Load the dataset
After running the first cell to load necessary libraries, next we'll load the dataset to be analyzed.

- Using pandas, load the dataset and save it as lyrics.
- Inspect the first few rows.
- Use .info() to determine how many rows there are, check for missing values, and check the variable types.

*If you want to inspect the lyrics, you can find them in the file tree. Click on the Jupyter logo in the upper left corner and you can see all the files.*

In [ ]:
# Load dataset
lyrics = pd.read_csv("LYRICS FILE NAME HERE")

# Inspect the first few rows
lyrics.head()

In [ ]:
# Get info about the DataFrame
lyrics.info()